In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
df_test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
df_sub  = pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")
df_sub.head()

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
pid=df_test['PassengerId']

In [ ]:
df_train=df_train.drop(columns=['PassengerId','Name'],axis=1)
df_test=df_test.drop(columns=['PassengerId','Name'],axis=1)
df_train.head()

In [ ]:
df_train.isna().sum()

In [ ]:
df_test.isna().sum()

In [ ]:
print(df_train['CryoSleep'].value_counts())
print(df_train['CryoSleep'].mode())

In [ ]:
print(df_train['VIP'].value_counts())
print(df_train['VIP'].mode())

In [ ]:
print(df_train['HomePlanet'].value_counts())
print(df_train['HomePlanet'].mode())

In [ ]:
print(df_train['Destination'].value_counts())
print(df_train['Destination'].mode())

In [ ]:
def clean_cat(data):
    data['HomePlanet'] =data['HomePlanet'].fillna('Earth')             ##  common value is (Earth)
    data['HomePlanet'] =data['HomePlanet'].map({'Europa':0 , 'Earth':1 , 'Mars':2})
    data['Destination']=data['Destination'].fillna("TRAPPIST-1e")     ## common value is TRAPPIST-1e 
    data['Destination']=data['Destination'].map({'TRAPPIST-1e':0 , 'PSO J318.5-22':1 ,  '55 Cancri e':2})
    data['VIP']        =data["VIP"].fillna(False).astype(int)         ## common value is False
    data['CryoSleep']  =data['CryoSleep'].fillna(False).astype(int)   ## common value is False
    return data

In [ ]:
def clean_num(data):
    for col in data.columns :
        if(data[col].dtype == 'float64'):
            data[col]=data[col].fillna(data[col].mean()) 
    return data 

In [ ]:
df_train=clean_cat(df_train)
df_train=clean_num(df_train)
df_test=clean_cat(df_test)
df_test=clean_num(df_test)
df_train.head()

In [ ]:
def cabin(cabin):
    try:
        return cabin.split('/')[0]
    except:
        return np.NaN

In [ ]:
df_train["Cabin"].fillna("F",inplace=True)
df_test["Cabin"].fillna("F",inplace=True)
df_train[df_train["Cabin"]=='F']

In [ ]:
df_train["Cabin"]=df_train["Cabin"].apply(lambda x : cabin(x))
df_test["Cabin"]=df_test["Cabin"].apply(lambda x : cabin(x))

In [ ]:
df_train['Cabin']=df_train['Cabin'].map({'A':0 , 'B':1 , 'C':2,'D':3 , 'E':4 , 'F':5,'T':6,'U':7})

In [ ]:
df_test['Cabin']=df_test['Cabin'].map({'A':0 , 'B':1 , 'C':2,'D':3 , 'E':4 , 'F':5,'T':6,'U':7})

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
model = RandomForestClassifier() 

In [ ]:
y_train=df_train['Transported']
x_train = df_train.drop(['Transported'] ,axis=1)
x_test =  df_test

In [ ]:
x_train['Cabin']=x_train['Cabin'].fillna(5.0)
x_test['Cabin']=x_test['Cabin'].fillna(5.0)

In [ ]:
model.fit(x_train,y_train)

In [ ]:
y_test = model.predict(x_test)

In [ ]:
sample = {'PassengerId': pid, 'Transported': y_test}
sample = pd.DataFrame.from_dict(sample)
sample.to_csv("submission.csv",index=False)
sample.head()